In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import html
import scrapy
from time import sleep
import urllib3
import json
from selenium import webdriver
import random

In [4]:
def mexico_puebla(link):
    mexico_puebla = pd.DataFrame()
    links_abiertos = []
    titulos_abiertos = []
    pdfs_proyectos = []
    descripciones = []

    driver = webdriver.Firefox()
    driver.get(link)

    metas = driver.find_elements_by_xpath('//h4[@class="entry-title"]/a')

    for meta in metas:
        link = meta.get_attribute('href') 
        if 'programa' not in link:
            links_abiertos.append(link)
            titulos_abiertos.append(meta.text)  
                  

    driver.quit()

    n = 0
    while n < len(links_abiertos):
        for link_abierto in links_abiertos:
            documentos = []
            desc = ''
            
            driver = webdriver.Firefox()
            driver.get(link_abierto)

            docs = driver.find_elements_by_xpath('//div[@class="post-wrapper-content"]//a')
            try:
                desc = driver.find_element_by_xpath('//div[@class="post-wrapper-content"]').text
            except:
                desc = ''
            
            descripciones.append(desc)
            

            for doc in docs:
                pdf = doc.get_attribute('href')
                if 'google' not in pdf:
                    documentos.append(pdf)
            
            

            driver.quit()

            pdfs_proyecto = ''

            for doc in documentos:
                pdfs_proyecto = pdfs_proyecto+doc+', '

            pdfs_proyectos.append(pdfs_proyecto.strip(', '))
            n+=1


    mexico_puebla['Título'] = titulos_abiertos
    mexico_puebla['Estado'] = 'Abierto'
    mexico_puebla['Descripción'] = descripciones
    mexico_puebla['Link'] = links_abiertos
    mexico_puebla['Pdfs'] = pdfs_proyectos
    

    return mexico_puebla


In [11]:
estados = ['http://www.concytep.gob.mx/convocatorias-y-programas-abiertos/', 'http://www.concytep.gob.mx/convocatorias-cerradas-y-resultados/page/1/', 'http://www.concytep.gob.mx/convocatorias-cerradas-y-resultados/page/2/', 'http://www.concytep.gob.mx/convocatorias-cerradas-y-resultados/page/3/', 'http://www.concytep.gob.mx/convocatorias-cerradas-y-resultados/page/4/']

mexico_pueblaT = pd.DataFrame()

puebla1 = mexico_puebla(estados[0])
puebla2 = mexico_puebla(estados[1])
puebla3 = mexico_puebla(estados[2])
puebla4 = mexico_puebla(estados[3])
puebla5 = mexico_puebla(estados[4])
#len(pdfs_proyectos)

In [19]:
frames = [puebla1, puebla2, puebla3, puebla4, puebla5]

puebla = pd.concat(frames)
puebla.reset_index(drop=True, inplace=True)

In [29]:
puebla.to_excel('Mexico_Puebla.xlsx')